###this is to run under myenv

In [1]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import numpy as np
print(np.__version__)

3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]


2.13.0
1.24.3


In [2]:
from myTokenize import SyllableTokenizer
from myTokenize import WordTokenizer
def word_tokenize(text):
    tokenizer = SyllableTokenizer(engine="myWord")
    tokens = tokenizer.tokenize(text)
    return tokens

In [1]:
import pandas as pd
from myTokenize import WordTokenizer

def tokenize_burmese_file(input_file, text_column='text', output_file='tokenized_output.xlsx', join_tokens=True):
    """
    Tokenizes Burmese text from an input CSV or Excel file using SyllableTokenizer.
    
    Parameters:
        input_file (str): Path to the input file (.csv or .xlsx).
        text_column (str): Name of the column containing Burmese text.
        output_file (str): Path to save the output file.
        join_tokens (bool): If True, join tokens with space. If False, keep tokens as list.
    """
    # Define tokenizer
    tokenizer = WordTokenizer(engine="CRF")

    # Read input file
    if input_file.endswith('.xlsx'):
        df = pd.read_excel(input_file)
    elif input_file.endswith('.csv'):
        df = pd.read_csv(input_file)
    else:
        raise ValueError("Unsupported file format. Please use .csv or .xlsx")

    # Ensure the text column exists
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in the input file.")
    print(f"Shape of input data: {df.shape}")
    # Remove trailing white spaces from the text column (say column is 'text')
    df[text_column] = df[text_column].astype(str).str.strip()

    # Remove duplicate rows based on the text column (or entire row if you want)
    df = df.drop_duplicates(subset=[text_column])
    print(f"Shape after removing duplicates: {df.shape}")
    # Tokenize row by row
    if join_tokens:
        df['tokens'] = df[text_column].astype(str).apply(lambda x: ' '.join(tokenizer.tokenize(x)))
    else:
        df['tokens'] = df[text_column].astype(str).apply(lambda x: tokenizer.tokenize(x))
    
    print(df['tokens'].head())  # Display first few tokenized rows for verification

    # Save to output file
    if output_file.endswith('.xlsx'):
        df.to_excel(output_file, index=False)
    elif output_file.endswith('.csv'):
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
    else:
        raise ValueError("Output file must end with .csv or .xlsx")

    print(f"Tokenized file saved to: {output_file}")


In [2]:
import os
input_file = os.path.abspath(os.path.join('datasets','raw_news.csv'))
tokenize_burmese_file(input_file,text_column='Sentence', output_file='tokenized_rawnews.csv', join_tokens=True)

Shape of input data: (949, 1)
Shape after removing duplicates: (903, 1)
0    ယနေ့ ည မန္တလေး - လား ရှိုးလမ်း တွင် တွဲ ကား ချ...
1    ပြင် ဦးလွင် နယ်မြေ (၃ ) ၊မန္တလေး - လား ရှိုးလမ...
2    အသင်း ထံ သို့ ဆက်သွယ် အ ကူအညီ တောင်းခံ ချက် အ ...
3    အမျိုးသား (၂ ) ဦး ၏De a d Body တို့ ကို ပြည်သူ...
4    UCLA တက္ကသိုလ် က သိပ္ ပံ ပညာရှင် Willia m Low ...
Name: tokens, dtype: object
Tokenized file saved to: tokenized_rawnews.csv


In [ ]:
from myTokenize import WordTokenizer

text = "ကျန်းမာရေးဝန်ကြီးဌာန၏ သစ်လွင်သားဖွား ခန့်အပ်တာဝန်ပေးအပ်ခြင်းအခမ်းအနားကို ယနေ့ နံနက် (၉:၀၀)နာရီတွင် ရန်ကုန်မြို့ရှိ သူနာပြုတက္ကသိုလ်ခန်းမ၌ကျင်းပရာ ကျန်းမာရေးဝန်ကြီးဌာန၊ ပြည်ထောင်စုဝန်ကြီး ပါမောက္ခဒေါက်တာသက်ခိုင်ဝင်းနှင့် ရန်ကုန်တိုင်းဒေသကြီးအစိုးရအဖွဲ့၊ ဝန်ကြီးချုပ်ဦးစိုးသိန်းတို့မှ တက်ရောက် အမှာစကား ပြောကြားခဲ့ပါသည်။"
tokenizer = WordTokenizer(engine="myWord")
words = tokenizer.tokenize(text)
print(words)  

In [3]:
from myTokenize import WordTokenizer

tokenizer = WordTokenizer(engine="myWord")
words = tokenizer.tokenize("မြန်မာနိုင်ငံ။")
print(words)  # ['မြန်မာ', 'နိုင်ငံ', '။']

['မြန်မာ', 'နိုင်ငံ', '။']


In [15]:
import pandas as pd

input_file = os.path.abspath(os.path.join('datasets','tokenized_ner.csv'))
# Load your file (adjust file name and format as needed)
df = pd.read_csv(input_file)  # or pd.read_excel('your_file.xlsx')

# Make sure your text column is in string format
df['tokens'] = df['tokens'].astype(str)

# Add a new column with text length
df['text_length'] = df['tokens'].str.len()

# Find the longest text
longest_text = df.loc[df['text_length'].idxmax()]
print("Longest Text:")
print(longest_text['tokens'])
print("Length:", longest_text['text_length'])

# Find the 5 shortest texts
shortest_texts = df.nsmallest(5, 'text_length')
print("\n5 Shortest Texts:")
for i, row in shortest_texts.iterrows():
    print(f"- ({row['text_length']} chars) {row['tokens']}")

Longest Text:
ထို့ နောက် ပြည်ထောင်စု အမျိုးသား လွှတ်တော် ဒုတိယ ဥက္ကဋ္ဌ ဦးအေးသာ အောင် က ဂုဏ်ပြု အ မှာ စကား ပြော ကြား ပေး ခဲ့ ပြီး တိုင်းရင်းသား လူမျိုး များ ရေး ရာဝန်ကြီးဌာန ပြည်ထောင်စု ဝန်ကြီး နိုင်သက်လွင် က " ကျွန်တော် တို့ တိုင်းရင်းသား အခွင့် အရေး များ ကို ကာကွယ် စောင့် ရှောက် ခြင်း အပြင် တိုင်းရင်းသား တို့ ရဲ့ စကား ၊ စာပေ ၊ယဉ်ကျေး မှု ဓလေ့ထုံးစံ များ ၊ အမျိုးသား ရေး လက္ခဏာ များ မ ပျောက် မ ပျက်အောင် ထိန်း သိမ်းဆောင်ရွက် လျက်ရှိ ပါ ကြောင်း၊ တစ်ဖက် က လည်း ဒေသ ဖွံ့ဖြိုးရေး အ တွက် ဘယ်လို ဆောင်ရွက် ရ မယ် ဆို တဲ့ အစီအ အ မံ တွေ ၊ အတွေး အ ခေါ် အမြှော် အမြင် တွေ ကို လည်း ပံပိုးကူညီ ဆောင်ရွက် ပေး သွား မည် ဖြစ် ပါ ကြောင်း၊ တိုင်းရင်းသား လူမျိုး များ ရဲ့ နေ့ထူး နေ့မြတ် အခမ်းအနား များ ကို လည်း လွတ်လွတ်လပ်လပ်ခမ်းခမ်းနားနားကျင်းပ ခွင့် ရရှိ စေ ရေး နှင့် သက်ဆိုင်ရာ ပြည်နယ်/ တိုင်း ဒေသ ကြီး အစိုးရ အဖွဲ့ က လည်း လို အပ် တဲ့ ရံပုံငွေ ထောက်ပံ့ ကူညီ ပေး နိုင် ဖို့ လည်းညှို နှိုင်းဆောင်ရွက် ပေး လျက်ရှိ ပါ ကြောင်း၊ တိုင်းရင်းသား များ ရဲ့ လူမှု ဘဝ တိုးတက် ရေး နှင့် ယဉ်ကျေး မှု အ မွေအ နှစ် များ ထိန်း သိမ်း စ

In [ ]:
# import pandas as pd
# import re

# # Step 1: Load the CSV
# df = pd.read_csv("datasets/ner_raw_final.csv", header=None, names=["Sentence"])

# # Step 2: Clean only rows that start with SNT.xxx.xxx
# df["Sentence"] = df["Sentence"].apply(
#     lambda x: re.sub(r'^SNT\.\d+\.\d+\s*', '', str(x)) if str(x).startswith("SNT.") else x
# )

# # Step 3: Save to a new cleaned file
# df.to_csv("datasets/ner_raw_final_cleanedSNT.csv", index=False, header=False, encoding='utf-8-sig')

